In [1]:
import string
import re
import time
import pandas as pd
import numpy as np
import codecs, csv, sys
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
start_time = time.time()
question1 = []
question2 = []
qid1 = []
qid2 = []
id_ = []
is_duplicate = []
f = open('/Volumes/Barly/NLP/Project/5k_testing/Type3/data/raw/test.csv','r')
reader = csv.DictReader(f)
for row in reader:
    question1.append(row['question1'].lower())
    question2.append(row['question2'].lower())
    qid1.append( row['qid1'])
    qid2.append( row['qid2'])
    id_.append( row['index'])
    is_duplicate.append( row['is_duplicate'])

In [3]:
punctuation_list = list(string.punctuation)
wordnet_lemmatizer = WordNetLemmatizer()

In [4]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This','at','have', 'been', 'on', 'too', 'in', 'it']

In [5]:
def sentences_splitted_to_words(sentence, remove_selective_stopwords = True, lemma=True):
    
    
    sentence = re.sub(r"what's", "", sentence)
    sentence = re.sub(r"What's", "", sentence)
    sentence = re.sub(r"\'s", " ", sentence)
    sentence = re.sub(r"\'ve ", " have ", sentence)
    sentence = re.sub(r"can't", "cannot ", sentence)
    sentence = re.sub(r"n't", " not ", sentence)
    sentence = re.sub(r"I'm", "I am", sentence)
    sentence = re.sub(r" m ", " am ", sentence)
    sentence = re.sub(r"\'re", " are ", sentence)
    sentence = re.sub(r"\'d ", " would ", sentence)
    sentence = re.sub(r"\'ll ", " will ", sentence)
    sentence = re.sub(r"60k", " 60000 ", sentence)
    sentence = re.sub(r" e g ", " example ", sentence)
    sentence = re.sub(r" b g ", " bg ", sentence)
    sentence = re.sub(r"\0s", "0", sentence)
    sentence = re.sub(r" 9 11 ", "911", sentence)
    sentence = re.sub(r"e-mail", "email", sentence)
    sentence = re.sub(r"\s{2,}", " ", sentence)
    sentence = re.sub(r"quikly", "quickly", sentence)
    sentence = re.sub(r" usa ", " America ", sentence)
    sentence = re.sub(r" USA ", " America ", sentence)
    sentence = re.sub(r" u s ", " America ", sentence)
    sentence = re.sub(r" uk ", " England ", sentence)
    sentence = re.sub(r" UK ", " England ", sentence)
    sentence = re.sub(r"india", "India", sentence)
    sentence = re.sub(r"switzerland", "Switzerland", sentence)
    sentence = re.sub(r"china", "China", sentence)
    sentence = re.sub(r"chinese", "Chinese", sentence) 
    sentence = re.sub(r"imrovement", "improvement", sentence)
    sentence = re.sub(r"intially", "initially", sentence)
    sentence = re.sub(r"quora", "Quora", sentence)
    sentence = re.sub(r" dms ", "direct messages ", sentence)  
    sentence = re.sub(r"demonitization", "demonetization", sentence) 
    sentence = re.sub(r"actived", "active", sentence)
    sentence = re.sub(r"kms", " kilometers ", sentence)
    sentence = re.sub(r"KMs", " kilometers ", sentence)
    sentence = re.sub(r" cs ", " computer science ", sentence) 
    sentence = re.sub(r" upvotes ", " up votes ", sentence)
    sentence = re.sub(r" iPhone ", " phone ", sentence)
    sentence = re.sub(r"\0rs ", " rs ", sentence) 
    sentence = re.sub(r"calender", "calendar", sentence)
    sentence = re.sub(r"ios", "operating system", sentence)
    sentence = re.sub(r"gps", "GPS", sentence)
    sentence = re.sub(r"gst", "GST", sentence)
    sentence = re.sub(r"programing", "programming", sentence)
    sentence = re.sub(r"bestfriend", "best friend", sentence)
    sentence = re.sub(r"dna", "DNA", sentence)
    sentence = re.sub(r"III", "3", sentence) 
    sentence = re.sub(r"the US", "America", sentence)
    sentence = re.sub(r"Astrology", "astrology", sentence)
    sentence = re.sub(r"Method", "method", sentence)
    sentence = re.sub(r"Find", "find", sentence) 
    sentence = re.sub(r"banglore", "Bangalore", sentence)
    sentence = re.sub(r" J K ", " JK ", sentence)
    sentence = re.sub(r"[^A-Za-z0-9]", " ", sentence)
    
    sentence = ''.join(ch for ch in sentence if ch not in punctuation_list)
    #print "Step 1: ", sentence
    if remove_selective_stopwords:
        sentence = sentence.split(" ")
        sentence = [word for word in sentence if word not in  stop_words]
        sentence = " ".join(sentence)
        #print "Step 2: removed stopwords ", sentence
        
    if lemma:
        sentence = sentence.split(" ")
        sentence_lemma = [wordnet_lemmatizer.lemmatize(word) for word in sentence]
        sentence = " ".join(sentence_lemma)
        #print "Step 3: lemmatized", sentence
    return sentence

In [12]:
df_doc2vec = pd.DataFrame()
for c_id, id1 , id2 , q1 , q2, label in zip(id_, qid1, qid2, question1, question2, is_duplicate):
    questions = [q1, q2]
    try:
        cleaned_question = []
        for question in questions:
            cleaned_sentence = sentences_splitted_to_words(question, True, True)
            #print "cleaned_sentence: ", cleaned_sentence
            cleaned_question.append(cleaned_sentence)
            
        df_doc2vec_temp = pd.DataFrame({'index':[c_id],'qid1': [id1], 'qid2': [id2], 'question1': [q1], 'question2': [q2],\
                                        'cleaned_question1': [cleaned_question[0]], 'cleaned_question2': [cleaned_question[1]],\
                                        'is_duplicate': [label]})
        df_doc2vec = pd.concat([df_doc2vec, df_doc2vec_temp])
    except:
        print c_id
extracted_data_time = time.time()

print "start-time: ", start_time, "\nextracted_data_time: ", extracted_data_time , "\n\nTotal time: ",(extracted_data_time -start_time )

start-time:  1513331841.74 
extracted_data_time:  1513331850.26 

Total time:  8.52851295471


In [13]:
df_doc2vec.to_csv("/Volumes/Barly/NLP/Project/5k_testing/Type3/data/Preprocessed/preprocessed_test_data.csv", index=False)